In [ ]:
import re
import numpy as np
import matplotlib.pyplot as plt
import copy
import open3d as o3d

In [ ]:
def get_points(s):
    pattern = re.compile(r'user_message\|([0-9\.\,\-e]*,)')
    new_points = []
    for i in re.findall(pattern, s):
        i = i.split(',')[:-1]
        if len(i) == 0:
            continue
        new_points.append([float(x) for x in i])
    new_points = np.array(new_points)
    return new_points

In [ ]:
def visualize_pointcloud(pcd, show_draw=False):
    pcd = copy.deepcopy(pcd)
    o3d.visualization.draw_plotly([pcd])
    if show_draw:
        R = pcd.get_rotation_matrix_from_xyz((-np.pi / 2, 0, np.pi / 2))
        pcd.rotate(R, center=(0, 0, 0))
        o3d.visualization.draw([pcd], show_ui=True)

In [ ]:
with open('points.txt') as f:
    s = f.read()

In [ ]:
points = get_points(s)
points[:, 1] *= -1

pcl = o3d.geometry.PointCloud()
pcl.points = o3d.utility.Vector3dVector(points)

visualize_pointcloud(pcl)

In [ ]:
def point_cloud_to_occupancy_map(points, grid_size, resolution, z_range):
  (z_min, z_max) = z_range
  mask = (points[:, 2] >= z_min) & (points[:, 2] <= z_max)
  points = points[mask, :2]

  binary_grid = np.zeros(grid_size, dtype=float)
  for point in points:
    x, y = point
    grid_x = int(grid_size[0] / 2 + x / resolution)
    grid_y = int(grid_size[1] / 2 + y / resolution)
    if 0 <= grid_x < grid_size[0] and 0 <= grid_y < grid_size[1]:
      binary_grid[grid_x, grid_y] = 1

  binary_grid = binary_grid.T

  return binary_grid

In [ ]:
def binary_array_to_points(binary_array):
    points = []
    for z in range(len(binary_array)):
        for y in range(len(binary_array[z])):
            for x in range(len(binary_array[z][y])):
                if binary_array[z][y][x] == 1:
                    points.append((x, y, z))
    return points

In [ ]:
grid = []

length = 200

z_start = -2
z_end = 5
planes = 5
z_size = (z_end - z_start) / planes
for i in range(planes):
    z_min = z_start + z_size * i
    z_max = z_start + z_size * (i + 1)
    new_grid = point_cloud_to_occupancy_map(points, (length, length), 3, (z_min, z_max))
    grid.append(new_grid)

pos = binary_array_to_points(grid)
#pos = [(p[0], -p[1], p[2]) for p in pos]

# Plot the points where the binary array is 1
pcl = o3d.geometry.PointCloud()
pcl.points = o3d.utility.Vector3dVector(pos)

visualize_pointcloud(pcl, True)

In [ ]:
points.shape